In [1]:
from google.cloud import bigquery as bq
import os
import pandas as pd
import pandas_gbq as pgbq
import util_functions as util
import matplotlib.pyplot as plt

%matplotlib inline

service_credentials = 'Service_Credentials/big-query-horse-play-f37757d450b8.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_credentials

ga_data_client = bq.Client(project='bigquery-public-data')

In [3]:
ga_dataset_ref = util.get_dataset(ga_data_client, 'google_analytics_sample')

---------------------------------------
Dataset ID: google_analytics_sample
Friendly Name: None
Full ID: bigquery-public-data:google_analytics_sample
Labels: {}
Project: bigquery-public-data
Ref: DatasetReference('bigquery-public-data', 'google_analytics_sample')
---------------------------------------


In [6]:
query = (
"""
SELECT
    visitId,
    device.deviceCategory,
    SUM(totals.visits) AS Visits,
    SUM(totals.pageviews) AS PVs,
    SUM(totals.newVisits) AS NewVisits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170101`
GROUP BY
    visitId,
    device.deviceCategory
""")

In [7]:
df = pgbq.read_gbq(query, dialect='standard')

In [9]:
df.fillna(value="None", inplace=True)

In [16]:
df.dtypes

visitId            int64
deviceCategory    object
Visits             int64
PVs                int64
NewVisits         object
dtype: object

In [17]:
weather_query = (
"""
WITH WashingtonStations AS (
  SELECT weather.stn AS station_id, ANY_VALUE(station.name) AS name
  FROM `bigquery-public-data.noaa_gsod.stations` AS station
  INNER JOIN `bigquery-public-data.noaa_gsod.gsod2015` AS weather
  ON station.usaf = weather.stn
  WHERE station.state = 'WA' AND station.usaf != '999999'
  GROUP BY station_id
)
SELECT washington_stations.name,
  (SELECT COUNT(*)
   FROM `bigquery-public-data.noaa_gsod.gsod2015` AS weather
   WHERE washington_stations.station_id = weather.stn
   AND max >= 70) AS warm_days
FROM WashingtonStations AS washington_stations
ORDER BY warm_days DESC;
""")

wash_weather = pgbq.read_gbq(weather_query, dialect='standard')

In [18]:
wash_weather

,name,warm_days
0,TRI CITIES,200
1,YAKIMA AIR TERMINAL/MCALSR FI,199
2,MUNICIPAL AIRPORT,195
3,WALLA WALLA REGIONAL ARPT,185
4,PANGBORN MEM,184
5,EPHRATA MUNICIPAL AIRPORT,184
6,MOSES LAKE/GRANT CO,183
7,BOWERS FLD,179
8,OMAK,176
9,FELTS FLD,172
